
# 7 Chakras Festival 2026 — Exploratory Data Analysis (EDA)

Questo notebook analizza il file esportato dei biglietti e costruisce alcune statistiche utili (vendite, entrate, paesi/città, tipi di ticket, sconti, duplicati, ecc.).  
**Percorso predefinito del dataset:**

```
C:\Users\spina\Documents\Other_Codes\7chackras\Documenti\ticket2026.csv
```

> Nota: I dati contengono informazioni personali. Usa e condividi questo notebook con attenzione.


In [ ]:

# === Setup ===================================================================
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from collections import Counter

# Visual options
pd.set_option("display.max_columns", 120)
pd.set_option("display.width", 160)

# Percorso del file CSV (modifica se necessario)
CSV_PATH = r"C:\Users\spina\Documents\Other_Codes\7chackras\Documenti\ticket2026.csv"

# Cartella di output locale per file puliti / grafici
OUTPUT_DIR = os.path.join(os.path.dirname(CSV_PATH), "eda_outputs")
os.makedirs(OUTPUT_DIR, exist_ok=True)

print("CSV_PATH:", CSV_PATH)
print("OUTPUT_DIR:", OUTPUT_DIR)


## 1) Caricamento dati

In [ ]:

# Proviamo un read_csv robusto per caratteri speciali e virgolette
read_kwargs = dict(
    sep=",",
    quotechar='"',
    encoding="utf-8",
    engine="python",
    dtype=str,   # carichiamo come stringhe per evitare conversioni sbagliate
    skip_blank_lines=True
)

df_raw = pd.read_csv(CSV_PATH, **read_kwargs)
print("Shape iniziale:", df_raw.shape)
df_raw.head(2)


## 2) Pulizia colonne e normalizzazione tipi

In [ ]:

# 2.1) Normalizza i nomi colonna: rimuovi spazi, doppi spazi e uniforma duplicati
def normalize_col(c):
    # strip, collassa spazi, togli spazi finali e punteggiatura ridondante
    c = (c or "").strip()
    c = " ".join(c.split())  # collapse spaces
    return c

cols = [normalize_col(c) for c in df_raw.columns]

# Deduplica i nomi uguali aggiungendo un suffisso progressivo
seen = {}
new_cols = []
for c in cols:
    if c not in seen:
        seen[c] = 0
        new_cols.append(c)
    else:
        seen[c] += 1
        new_cols.append(f"{c}__{seen[c]}")  # es: "Phone number ...__1"

df = df_raw.copy()
df.columns = new_cols

print("Numero colonne:", len(df.columns))
print("Esempio colonne:\n", df.columns[:25].tolist())


In [ ]:

# 2.2) Conversioni utili di tipo

# Colonna data pagamento (diversi formati locali: "31/10/2025 - 07:57")
def parse_payment_date(s):
    if pd.isna(s): 
        return pd.NaT
    s = str(s).replace("–", "-").replace("—", "-")
    s = s.strip()
    # Prova formati più comuni italiani/europei
    for fmt in ["%d/%m/%Y - %H:%M", "%d/%m/%Y %H:%M", "%d/%m/%Y", "%d-%m-%Y %H:%M"]:
        try:
            return datetime.strptime(s, fmt)
        except ValueError:
            continue
    return pd.NaT

if "Payment Date" in df.columns:
    df["Payment_Date_parsed"] = df["Payment Date"].map(parse_payment_date)
else:
    df["Payment_Date_parsed"] = pd.NaT

# Converte campi numerici potenzialmente stringa in float
def to_num(x):
    if x is None or (isinstance(x,float) and np.isnan(x)) or str(x).strip()=="" or str(x).strip().lower()=="nan":
        return np.nan
    s = str(x).strip().replace("€","").replace(",","").replace(" ", "")
    try:
        return float(s)
    except:
        # prova con virgola decimale
        s2 = str(x).strip().replace("€","").replace(".", "").replace(",", ".").replace(" ", "")
        try:
            return float(s2)
        except:
            return np.nan

num_candidates = ["Order Total","Ticket Subtotal","Ticket Discount","Ticket Fee","Ticket Total","Price"]
for c in num_candidates:
    if c in df.columns:
        df[c+"_num"] = df[c].map(to_num)

# Normalizza email (minuscolo/strip) per dedupliche
for c in ["Attendee E-mail","Buyer E-Mail"]:
    if c in df.columns:
        df[c] = df[c].astype(str).str.strip().str.lower()

print(df[["Payment Date","Payment_Date_parsed"]].head(5))


In [ ]:

# Salva una copia "pulita" per usi futuri
clean_path = os.path.join(OUTPUT_DIR, "tickets_clean.csv")
df.to_csv(clean_path, index=False, encoding="utf-8")
print("Salvato:", clean_path)


## 3) Panoramica rapida

In [ ]:

n_rows, n_cols = df.shape
print(f"Righe (record biglietti): {n_rows:,}")
print(f"Colonne: {n_cols:,}")

# Campi chiave disponibili
key_fields = [c for c in [
    "Event Name","Order Number","Order Status","Payment Date","Payment_Date_parsed",
    "Attendee E-mail","Buyer E-Mail","Ticket Type","Ticket Code","Ticket ID",
    "Ticket Total_num","Order Total_num","Price_num","Discount Code",
    "Country of residence / Paese di residenza (Campi ticket holder)",
    "City of residence / Città di residenza (Campi ticket holder)"
] if c in df.columns]
print("\nCampi chiave:\n", key_fields)

df[key_fields].head(10)


## 4) Vendite & ricavi

In [ ]:

tot_tickets = len(df)
tot_revenue_ticket = df.get("Ticket Total_num", pd.Series(dtype=float)).sum()
tot_revenue_order = df.get("Order Total_num", pd.Series(dtype=float)).sum()
avg_ticket_price = df.get("Ticket Total_num", pd.Series(dtype=float)).mean()

print(f"Totale biglietti: {tot_tickets:,}")
print(f"Somma Ticket Total (€): {tot_revenue_ticket:,.2f}")
print(f"Somma Order Total (€):  {tot_revenue_order:,.2f}")
print(f"Prezzo medio per riga (Ticket Total): {avg_ticket_price:,.2f} €")

# Vendite per tipo di ticket
by_type = df.groupby("Ticket Type", dropna=False).agg(
    tickets=("Ticket Type","size"),
    revenue=("Ticket Total_num","sum"),
    avg_price=("Ticket Total_num","mean")
).sort_values(["revenue","tickets"], ascending=False)
by_type.head(20)


In [ ]:

# Grafico: ticket per tipo
counts = by_type["tickets"]
plt.figure(figsize=(10,5))
counts.plot(kind="bar")
plt.title("Conteggio biglietti per 'Ticket Type'")
plt.ylabel("Biglietti")
plt.tight_layout()
plt.show()


In [ ]:

# Timeline vendite: biglietti per giorno (basata su Payment_Date_parsed)
ts = df.dropna(subset=["Payment_Date_parsed"]).copy()
if not ts.empty:
    ts["date"] = ts["Payment_Date_parsed"].dt.date
    daily = ts.groupby("date").size()

    plt.figure(figsize=(10,4))
    daily.plot(kind="line", marker="o")
    plt.title("Biglietti venduti per giorno")
    plt.xlabel("Data")
    plt.ylabel("Biglietti")
    plt.tight_layout()
    plt.show()

    # Cumulata
    plt.figure(figsize=(10,4))
    daily.cumsum().plot(kind="line", marker="o")
    plt.title("Biglietti cumulati nel tempo")
    plt.xlabel("Data")
    plt.ylabel("Cumulato biglietti")
    plt.tight_layout()
    plt.show()
else:
    print("Nessuna data di pagamento valida per costruire una timeline.")


## 5) Provenienza geografica

In [ ]:

country_col = "Country of residence / Paese di residenza (Campi ticket holder)"
city_col    = "City of residence / Città di residenza (Campi ticket holder)"

if country_col in df.columns:
    by_country = df.groupby(country_col, dropna=False).size().sort_values(ascending=False)
    print("Top paesi:\n", by_country.head(20))
    plt.figure(figsize=(10,4))
    by_country.head(15).plot(kind="bar")
    plt.title("Top paesi (prime 15 voci)")
    plt.ylabel("Biglietti")
    plt.tight_layout()
    plt.show()

if city_col in df.columns:
    by_city = df.groupby(city_col, dropna=False).size().sort_values(ascending=False)
    print("Top città:\n", by_city.head(20))
    plt.figure(figsize=(10,4))
    by_city.head(15).plot(kind="bar")
    plt.title("Top città (prime 15 voci)")
    plt.ylabel("Biglietti")
    plt.tight_layout()
    plt.show()


## 6) Codici sconto & scontistica

In [ ]:

if "Discount Code" in df.columns:
    has_disc = df["Discount Code"].fillna("").str.strip() != ""
    n_disc = has_disc.sum()
    print(f"Righe con codice sconto: {n_disc} ({n_disc/len(df)*100:.1f}%)")

    disc_counts = df.loc[has_disc, "Discount Code"].value_counts()
    print("\nFrequenza per codice sconto:\n", disc_counts)

    plt.figure(figsize=(8,4))
    disc_counts.plot(kind="bar")
    plt.title("Frequenza codici sconto")
    plt.ylabel("Occorrenze")
    plt.tight_layout()
    plt.show()

    # Sconto medio (se 'Ticket Discount_num' presente)
    if "Ticket Discount_num" in df.columns:
        avg_disc = df.loc[has_disc, "Ticket Discount_num"].mean()
        print(f"\nSconto medio (solo righe con sconto): {avg_disc:.2f} €")
else:
    print("Colonna 'Discount Code' non presente.")


## 7) Check-in (se disponibili)

In [ ]:

# Alcune esportazioni hanno colonne: 'Checked-in', 'Check-ins', 'Check-outs'
for c in ["Checked-in","Check-ins","Check-outs"]:
    if c in df.columns:
        print(c, "— esempi:", df[c].dropna().unique()[:10])

if "Checked-in" in df.columns:
    checked = df["Checked-in"].str.strip().str.lower()
    counts = checked.value_counts(dropna=False)
    print("\nStato 'Checked-in':\n", counts)


## 8) Duplicati & qualità dati

In [ ]:

# Potenziali duplicati per email partecipante
dup_att = []
if "Attendee E-mail" in df.columns:
    vc = df["Attendee E-mail"].value_counts()
    dup_att = vc[vc>1]
    print("Email (Attendee) con più di un record:", len(dup_att))
    display(dup_att.head(20))

# Potenziali duplicati per Order Number
if "Order Number" in df.columns:
    vc_ord = df["Order Number"].value_counts()
    multi_in_order = vc_ord[vc_ord>1]
    print("\nOrdini con più di una riga:", len(multi_in_order))
    display(multi_in_order.head(20))

# Campi mancanti su colonne chiave
key_miss_cols = [c for c in ["Attendee E-mail","Buyer E-Mail","Order Number","Ticket Type","Ticket Total_num"] if c in df.columns]
missing_summary = df[key_miss_cols].isna().mean().sort_values(ascending=False)
print("\n% Valori mancanti su campi chiave:\n", (missing_summary*100).round(1))

# Righe sospette (prezzo zero ma stato 'Paid', ecc.)
sus = df[(df.get("Ticket Total_num", pd.Series(dtype=float)).fillna(0)==0) & (df.get("Order Status", "").astype(str).str.lower()=="paid")]
print(f"\nRighe con Ticket Total zero ma ordine 'Paid': {len(sus)}")
if len(sus)>0:
    display(sus[["Order Number","Order Status","Ticket Total","Ticket Total_num","Discount Code"]].head(10))


## 9) Esportazioni riassuntive

In [ ]:

# Salva alcune tabelle utili
exports = {}

if "Ticket Type" in df.columns:
    exports["by_type.csv"] = df.groupby("Ticket Type", dropna=False).agg(
        tickets=("Ticket Type","size"),
        revenue=("Ticket Total_num","sum"),
        avg_price=("Ticket Total_num","mean")
    ).sort_values(["revenue","tickets"], ascending=False)

country_col = "Country of residence / Paese di residenza (Campi ticket holder)"
if country_col in df.columns:
    exports["by_country.csv"] = df.groupby(country_col, dropna=False).size().to_frame("tickets").sort_values("tickets", ascending=False)

for name, tab in exports.items():
    outp = os.path.join(OUTPUT_DIR, name)
    tab.to_csv(outp, encoding="utf-8")
    print("Esportato:", outp)

print("\nFatto!")



---

### Suggerimenti
- Se alcune colonne hanno lo stesso nome (es. campi ripetuti per più ticket holder), sono state rinominate con suffissi `__1`, `__2`, ecc.
- Per analisi avanzate (es. validazione Codice Fiscale), puoi aggiungere celle dedicate.
- Per una segmentazione per **fase prezzo** puoi usare `Ticket Type` (contiene "PHASE 0", "PHASE 1", ecc.).
